# SD202 TP2 - Normalization and SQL

*Author: Benjamin Ternot*

The objectives of this TP are the following:

1. Apply normalization 1NF -> 2NF -> 3NF
2. Perform SQL queries on the normalized database

In this lab, we are going to use a database containing wine information related to 'production' and 'sales'. 

Production <---> Wine <---> Sales


---

### Working with db files in Jupyter
- Python provides an interface for SQLite through the *sqlite3* module
- The **`%%sql`** magic builds upon it (and other tools) to enable the usage of SQL commands within a Jupyter Notebook as in common SQL clients.
- Before proceeding, make sure that **`wine.db`** is in the same path as this notebook.
  - If **`wine.db`** is not in the same path, an empty **`wine.db`** file will be created, resulting in errors in later steps of the TP.
- The SQLite module in Python commits transactions automatically, this means that any change in the DB is immediately written to the file, e.g. creating/deleting tables.
  -  For this reason, it is recommended to have a backup of **`wine.db`** as it is provided in the course's website.

---

**`wine.db`** contains the following unnormalized tables:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

From wikipedia:

__Cru__: Often used to indicate a specifically named and legally defined vineyard or ensemble of vineyards and the vines "which grow on [such] a reputed terroir; by extension of good quality." The term is also used to refer to the wine produced from such vines.


<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


We need to prepare the SQL environment:

In [1]:
import sqlite3

In [2]:
def printSchema(connection):
    ### Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [3]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)           # An usefull way to viualize the content of the database

Database schema:
CLIENTS:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
LOC:
  0: LIEU(TEXT)
  1: REGION(TEXT)
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(NUM)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
PRODUCERS:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
PRODUCTIONS:
  0: NV(NUM)
  1: NP(NUM)
  2: QTE(NUM)
SALES:
  0: NV(NUM)
  1: NB(NUM)
  2: QTE(NUM)
  3: DATES(NUM)
  4: LIEU(TEXT)
WINES1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
WINES2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)


We recommend inline __%sql__ as an alternative to sqlite3 package

In [4]:
%load_ext sql
%sql sqlite:///wine.db

Now, we can see the content of the tables using SQL queries:

In [5]:
%sql SELECT DISTINCT NV, CRU, MILL, DEGRE FROM MASTER1;

*  sqlite:///wine.db
Done.


NV,CRU,MILL,DEGRE
None,None,None,None
1,Mercurey,1980,11.5
2,Julienas,1974,11.3
3,Savigny les Beaunes,1978,12.1
4,Mercurey,1980,10.9
5,Pommard,1976,11.7
6,Mercurey,1981,11.2
7,Grands Echezeaux,1968,11.7
8,Cotes de Beaune Villages,1975,12.3
9,Chapelle Chambertin,1973,11.9


# PART I: Database normalization

The first task on this TP is the normalization of the wine data. In its current state both tables Master1 and Master2 are in the First Normal Form (1NF) and suffer from data redundancy. 

__1.1__ Given the set of the following FDs:

NV -> CRU, DEGRE, MILL

NP -> NOM, PRENOM, REGION

NV, NP -> QTE

Convert table Master1 to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.

* Explain your answer
* Describe the schema of new tables and how they relate

**Answer:**  

<blockquote>
From the FDs, we can identify the following primary keys:  
    
- NV (from FD 1)
- NP (from FD 2)

Based on the FDs, we can split the `Master1` table into three tables to achieve the third normalization form (3NF):

- Table `Wines` (Primary key: NV):
    - NV (primary key)
    - CRU
    - DEGRE
    - MILL  
    This table contains the information related to NV, as indicated by FD 1.  


- Table `Producers` (Primary key: NP):
    - NP (primary key)
    - NOM
    - PRENOM
    - REGION  
    This table contains the information related to NP, as indicated by FD 2.  


- Table `Productions` (Primary key: NV, NP):
     - NV (foreign key)
     - NP (foreign key)
     - QTE  
    This table represents the relationship between NV and NP, as indicated by FD 3.  

The relationships between the tables are as follows:

- `Wines` and `Producers` have a one-to-many relationship based on the NV foreign key in the `Wines` table and the NP foreign key in the `Producers` table. This relationship represents the association between a wine (identified by NV) and its producer (identified by NP).

- `Productions` table represents the relationship between NV and NP, indicating the productions for a specific wine (identified by NV) by a specific producer (identified by NP). The `NV` and `NP` columns in the `Productions` table are foreign keys referencing the primary keys in the `Wines` and `Producers` tables, respectively.


In the new schema, the `Wines` and `Producers` are in the Third Normal Form (3NF) since there are no partial dependencies or transitive dependencies. The `Productions` is also in 3NF since it only contains the necessary attributes for the relationship.
</blockquote>

__1.2__ Given the set of the following FDs:

NV -> CRU, DEGRE, MILL

NB -> NOM, PRENOM, TYPE

NV, NB -> DATES, LIEU, QTE

LIEU -> REGION 

Convert table Master2 to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable. 
* Explain your answer
* Describe the schema of new tables and how they relate

**Answer:**  
<blockquote>
From the FDs, we can identify the following primary keys:  
    
- NV (from FD 1)
- NP (from FD 2)

Based on the FDs, we can split the `Master2` table into 4 tables to achieve the third normalization form (3NF):

- Table `Wines` (Primary key: NV):
    - NV (primary key)
    - CRU
    - DEGRE
    - MILL  
    This table contains the information related to NV, as indicated by FD 1.  


- Table `Clients` (Primary key: NB):
    - NB (primary key)
    - NOM
    - PRENOM
    - TYPE  
    This table contains the information related to NP, as indicated by FD 2.  


- Table `Sales` (Primary key: NV, NB):
     - NV (foreign key)
     - NB (foreign key)
     - DATES
     - LIEU (foreign key)
     - QTE  
    This table represents the relationship between NV and NB, as indicated by FD 3.


- Table `Loc` (Primary key: LIEU):
     - LIEU (primary key)
     - REGION  
    This table represents the relationship between LIEU and REGION, as indicated by FD 4. It contains information about the location and corresponding region.

In this 3NF schema, the `Wines` and `Clients` tables are in the 3NF since they have single-valued attributes and primary keys. The `Sales` table is also in 3NF as it only contains the necessary attributes for the relationship between NV and NB. Similarly, the `Loc` table is in 3NF as it contains the necessary information related to LIEU and REGION.

The relationships between the tables are as follows:
- `Wines` and `Sales` have a one-to-many relationship based on the NV foreign key.
- `Clients` and `Sales` have a one-to-many relationship based on the NB foreign key.
- `Loc` and `Sales` have a one-to-one relationship based on the LIEU foreign key.

</blockquote>

Once you have defined the 2NF or 3NF (as applicable) we need to split the data into new tables. A table can be created from the result of a query. In the following example we will create a new table "dummy" to store the different values of alcohol content.

In [6]:
%sql DROP TABLE IF EXISTS dummy;

# Create dummy table
%sql CREATE TABLE dummy AS \
SELECT DISTINCT DEGRE \
FROM MASTER1;

print("\nContent of the database")
printSchema(conn)

print("\nContent of dummy")
%sql SELECT * FROM dummy

*  sqlite:///wine.db
Done.
*  sqlite:///wine.db
Done.

Content of the database
CLIENTS:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
LOC:
  0: LIEU(TEXT)
  1: REGION(TEXT)
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(NUM)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
PRODUCERS:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
PRODUCTIONS:
  0: NV(NUM)
  1: NP(NUM)
  2: QTE(NUM)
SALES:
  0: NV(NUM)
  1: NB(NUM)
  2: QTE(NUM)
  3: DATES(NUM)
  4: LIEU(TEXT)
WINES1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
WINES2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
dummy:
  0: DEGRE(NUM)

Content of dummy
*  sqlite:///wine.db
Done.


DEGRE
None
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8


In [7]:
# Remove dummy table
%sql DROP TABLE IF EXISTS dummy;

*  sqlite:///wine.db
Done.


__1.3__ Create the new tables from Master1:

In [8]:
# Write corresponding code here
%sql DROP TABLE IF EXISTS WINES1;
%sql DROP TABLE IF EXISTS PRODUCERS;
%sql DROP TABLE IF EXISTS PRODUCTIONS;


# Create WINES1 table
%sql CREATE TABLE WINES1 AS \
SELECT DISTINCT NV, CRU, DEGRE, MILL \
FROM MASTER1;

# Create PRODUCERS table
%sql CREATE TABLE PRODUCERS AS \
SELECT DISTINCT NP, NOM, PRENOM, REGION \
FROM MASTER1;

# Create PRODUCTIONS table
%sql CREATE TABLE PRODUCTIONS AS \
SELECT DISTINCT NV, NP, QTE \
FROM MASTER1;


print("\nContent of the database")
printSchema(conn)

*  sqlite:///wine.db
Done.
*  sqlite:///wine.db
Done.
*  sqlite:///wine.db
Done.
*  sqlite:///wine.db
Done.
*  sqlite:///wine.db
Done.
*  sqlite:///wine.db
Done.

Content of the database
CLIENTS:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
LOC:
  0: LIEU(TEXT)
  1: REGION(TEXT)
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(NUM)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
PRODUCERS:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
PRODUCTIONS:
  0: NV(NUM)
  1: NP(NUM)
  2: QTE(NUM)
SALES:
  0: NV(NUM)
  1: NB(NUM)
  2: QTE(NUM)
  3: DATES(NUM)
  4: LIEU(TEXT)
WINES1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
WINES2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)


__1.4__ Create the new tables from Master2:

In [9]:
# Write corresponding code here
%sql DROP TABLE IF EXISTS WINES2;
%sql DROP TABLE IF EXISTS CLIENTS;
%sql DROP TABLE IF EXISTS SALES;
%sql DROP TABLE IF EXISTS LOC;


# Create WINES2 table
%sql CREATE TABLE WINES2 AS \
SELECT DISTINCT NV, CRU, DEGRE, MILL \
FROM MASTER2;

# Create CLIENTS table
%sql CREATE TABLE CLIENTS AS \
SELECT DISTINCT NB, NOM, PRENOM, TYPE \
FROM MASTER2;

# Create SALES table
%sql CREATE TABLE SALES AS \
SELECT DISTINCT NV, NB, QTE, DATES, LIEU \
FROM MASTER2;

# Create LOC table
%sql CREATE TABLE LOC AS \
SELECT DISTINCT LIEU, REGION \
FROM MASTER2;


print("\nContent of the database")
printSchema(conn)

*  sqlite:///wine.db
Done.
*  sqlite:///wine.db
Done.
*  sqlite:///wine.db
Done.
*  sqlite:///wine.db
Done.
*  sqlite:///wine.db
Done.
*  sqlite:///wine.db
Done.
*  sqlite:///wine.db
Done.
*  sqlite:///wine.db
Done.

Content of the database
CLIENTS:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
LOC:
  0: LIEU(TEXT)
  1: REGION(TEXT)
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(NUM)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
PRODUCERS:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
PRODUCTIONS:
  0: NV(NUM)
  1: NP(NUM)
  2: QTE(NUM)
SALES:
  0: NV(NUM)
  1: NB(NUM)
  2: QTE(NUM)
  3: DATES(NUM)
  4: LIEU(TEXT)
WINES1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
WINES2:
  0: NV(NUM)
  1: CRU(TEX

# PART II: SQL QUERIES

In the second part of this TP you will create SQL queries to retrieve information from the database.

__2.1__ What are the different types of clients (buveurs) by volume of purchases?

In [10]:
%%sql
SELECT DISTINCT TYPE AS Type_of_buveurs
FROM CLIENTS;

*  sqlite:///wine.db
Done.


Type_of_buveurs
petit
moyen
gros
None


__2.2__ What regions produce Pommard or Brouilly?

In [11]:
%%sql
SELECT DISTINCT CRU, REGION
FROM WINES1 AS w
    JOIN PRODUCTIONS AS p1 ON w.NV = p1.NV AND (w.CRU = "Pommard" OR w.CRU = "Brouilly")
    JOIN PRODUCERS AS p2 ON (p1.NP = p2.NP)
ORDER BY REGION ASC, CRU ASC;

*  sqlite:///wine.db
Done.


CRU,REGION
Brouilly,Bourgogne
Pommard,Bourgogne
Pommard,Rhone


__2.3__ What regions produce Pommard and Brouilly?

In [12]:
%%sql
SELECT REGION
FROM
    (SELECT DISTINCT CRU, REGION
    FROM WINES1 AS w
        JOIN PRODUCTIONS AS p1 ON w.NV = p1.NV AND (w.CRU = "Pommard" OR w.CRU = "Brouilly")
        JOIN PRODUCERS AS p2 ON (p1.NP = p2.NP))
GROUP BY REGION
HAVING COUNT(CRU) = 2
ORDER BY REGION ASC;

*  sqlite:///wine.db
Done.


REGION
Bourgogne


__2.4__ Get the number of wines bought by CRU and Millésime

In [13]:
%%sql
SELECT CRU, MILL, SUM(QTE) AS Number_of_wines_bought
FROM WINES2 AS w
    JOIN SALES AS s ON w.NV = s.NV
GROUP BY CRU, MILL
HAVING SUM(QTE) IS NOT NULL
ORDER BY CRU ASC, MILL ASC;

*  sqlite:///wine.db
Done.


CRU,MILL,Number_of_wines_bought
Arbois,1980,8
Auxey Duresses,1914,80
Beaujolais Primeur,1983,7
Beaujolais Villages,1975,10
Beaujolais Villages,1976,120
Beaujolais Villages,1978,130
Beaujolais Villages,1979,520
Chapelle Chambertin,1973,30
Chateau Corton Grancey,1980,4
Chenas,1984,1


__2.5__ Retrieve the wine number (NV) of wines produced by more than three producers

In [14]:
%%sql
SELECT NV, COUNT(DISTINCT(NP)) AS Number_of_producers
FROM PRODUCTIONS
GROUP BY NV
HAVING COUNT(DISTINCT(NP)) > 3 AND NV IS NOT NULL
ORDER BY NV ASC;

*  sqlite:///wine.db
Done.


NV,Number_of_producers
45,5
78,5
89,4
98,5


__2.6__ Which producers have not produced any wine?

In [15]:
%%sql
SELECT p.NP, NOM, PRENOM
FROM PRODUCERS AS p
    JOIN PRODUCTIONS AS pr ON p.NP = pr.NP
GROUP BY p.NP, NOM, PRENOM
HAVING SUM(QTE) IS NULL
ORDER BY p.NP ASC;

*  sqlite:///wine.db
Done.


NP,NOM,PRENOM
3,Six,Paul
6,Marmagne,Bernard
8,Lioger d'Harduy,Gabriel
16,Barbin,Bernard
17,Faiveley,Guy
18,Tramier,Jean
19,Dupaquier,Roger
20,Lamy,Jean
21,Cornu,Edmond
26,Violot,Gilbert


__2.7__ What clients (buveurs) have bought at least one wine from 1980?

In [16]:
%%sql
SELECT DISTINCT c.NB, NOM, PRENOM
FROM WINES2 AS w
    JOIN SALES AS s ON w.NV = s.NV AND w.MILL = 1980
    JOIN CLIENTS AS c ON s.NB = c.NB
ORDER BY c.NB ASC;

*  sqlite:///wine.db
Done.


NB,NOM,PRENOM
2,Artaud,Antonin
8,Aragon,Louis
44,Gide,Andre
45,Giono,Jean
50,Lautreamont,None
61,Mallarme,Stephane


__2.8__ What clients (buveurs) have NOT bought any wine from 1980?

In [17]:
%%sql
SELECT NB, NOM, PRENOM
FROM CLIENTS
WHERE NB NOT IN
    (SELECT DISTINCT c.NB
    FROM WINES2 AS w
        JOIN SALES AS s ON w.NV = s.NV AND w.MILL = 1980
        JOIN CLIENTS AS c ON s.NB = c.NB)
ORDER BY NB ASC;

*  sqlite:///wine.db
Done.


NB,NOM,PRENOM
1,Aristote,None
3,Aron,Raymond
4,Apollinaire,Guillaume
5,Audiberti,Jacques
6,Arrabal,Fernando
7,Anouilh,Jean
9,Ajar,Emile
10,Andersen,Yann
11,Breton,Andre
12,Bataille,Georges


__2.9__ What clients (buveurs) have bought ONLY wines from 1980?

In [18]:
%%sql
SELECT DISTINCT c.NB, NOM, PRENOM
FROM WINES2 AS w
    JOIN SALES AS s ON w.NV = s.NV AND w.MILL = 1980
    JOIN CLIENTS AS c ON s.NB = c.NB
WHERE c.NB NOT IN
    (SELECT DISTINCT c.NB
    FROM WINES2 AS w
        JOIN SALES AS s ON w.NV = s.NV AND w.MILL <> 1980
        JOIN CLIENTS AS c ON s.NB = c.NB)
ORDER BY c.NB ASC;

*  sqlite:///wine.db
Done.


NB,NOM,PRENOM
44,Gide,Andre
45,Giono,Jean
50,Lautreamont,None


__2.10__ List all wines from 1980

In [19]:
%%sql
SELECT *
FROM WINES2
WHERE MILL = 1980
ORDER BY NV ASC;

*  sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL
1,Mercurey,11.5,1980
4,Mercurey,10.9,1980
16,Meursault,12.1,1980
20,Cote de Brouilly,12.1,1980
26,Chateau Corton Grancey,None,1980
28,Volnay,11,1980
43,Fleurie,11.4,1980
74,Arbois,12,1980
78,Etoile,12,1980
79,Seyssel,11,1980


__2.11__ What are the wines from 1980 bought by NB=2?

In [20]:
%%sql
SELECT w.NV, s.NB, s.DATES, s.LIEU, s.QTE, w.CRU, w.MILL, w.DEGRE
FROM WINES2 AS w
    JOIN SALES AS s ON w.MILL = 1980 AND w.NV = s.NV AND s.NB = 2
ORDER BY w.NV ASC;

*  sqlite:///wine.db
Done.


NV,NB,DATES,LIEU,QTE,CRU,MILL,DEGRE
1,2,1977-11-02,BORDEAUX,33,Mercurey,1980,11.5


__2.12__ What clients (buveurs) have bought ALL the wines from 1980?

In [21]:
%%sql
SELECT c.NB, c.NOM, c.PRENOM
FROM WINES2 AS w
    JOIN SALES AS s ON w.MILL = 1980 AND s.NV = w.NV
    JOIN CLIENTS AS c ON c.Nb = s.NB
GROUP BY c.NB, c.NOM, c.PRENOM
HAVING COUNT(DISTINCT(w.NV)) =
    (SELECT COUNT(DISTINCT(NV))
    FROM WINES2
    WHERE MILL = 1980)
ORDER BY c.NB ASC;

*  sqlite:///wine.db
Done.


NB,NOM,PRENOM
44,Gide,Andre
